In [1]:
import numpy as np
import torch
import sys
sys.path.append("..")
from scipy import stats
from scipy import io
import time
import sys
sys.path.append("..")
from EnsemblePursuitModule.EnsemblePursuitNumpyFast import EnsemblePursuitNumpyFast
from scipy.ndimage import gaussian_filter, gaussian_filter1d 
import matplotlib.pyplot as plt

In [2]:
#Load in the data
#Set data path to your own path
spks= np.load('/home/flora/Documents/spks.npy')
print('Shape of the data matrix, timepoints by neurons:',spks.shape)
mov    = np.load('/home/flora/Documents/mov.npy')# these are the visual stimuli shown
iframe = np.load('/home/flora/Documents/iframe.npy') # iframe[n] is the microscope frame for the image frame n


Shape of the data matrix, timepoints by neurons: (18795, 30766)


In [3]:
# align stimuli to spks (since stimuli are triggered by microscope it's easy)
dt = 1 # time offset between stimulus presentation and response
ivalid = iframe+dt<spks.shape[-1] # remove timepoints outside the valid time range
iframe = iframe[ivalid]
mov = mov[:, :, ivalid]
S = spks[:, iframe+dt]
del spks
S = stats.zscore(S, axis=1) # z-score the neural activity before doing anything

In [4]:
options_dict={'seed_neuron_av_nr':100,'min_assembly_size':8}
nr_of_components=200
start=time.time()
ep_np=EnsemblePursuitNumpyFast(n_ensembles=nr_of_components,lambd=0.01,options_dict=options_dict)
U,V=ep_np.fit_transform(S)
end=time.time()
print(end-start)
print('Shape of U and V',U.shape,V.shape)
np.save('V.npy',V)
np.save('U.npy',U)

KeyboardInterrupt: 

In [ ]:
from scipy.stats import zscore
Sp = V.T
Sp = zscore(Sp, axis=1)
ly, lx, nstim = mov.shape

NN, NT = S.shape 

X = np.reshape(mov, [-1, NT]) # reshape to Npixels by Ntimepoints
X = X-0.5 # subtract the background
X = np.abs(X) # does not matter if a pixel is black (0) or white (1)
X = stats.zscore(X, axis=1)/NT**.5  # z-score each pixel separately
npix = X.shape[0]

lam = .0001
ncomps = Sp.shape[0]
B0 = np.linalg.solve((X @ X.T + lam * np.eye(npix)),  (X @ Sp.T)) # get the receptive fields for each neuron

B0 = np.reshape(B0, (ly, lx, ncomps))
B0 = gaussian_filter(B0, [.5, .5, 0]) # smooth each receptive field a little

plt.figure(figsize=(18, 8))
rfmax = np.max(B0)
for j in range(200):
    plt.subplot(10,20,j+1)
    rf = B0[:,:,j]
    # rfmax = np.max(np.abs(rf))
    plt.imshow(rf, aspect='auto', cmap = 'bwr', vmin = -rfmax, vmax = rfmax) # plot the receptive field for each neuron
    #plt.title('PC %d'%(1+j))
    plt.axis('off')
    
plt.show()    

In [ ]:
proc = np.load('/home/flora/Documents/cam1_TX39_2019_05_31_1_proc_resampled.npy', allow_pickle=True).item()
motSVD = proc['motSVD'][:,iframe+dt]
print(motSVD.shape)
beh = stats.zscore(beh, axis=1)/NT**.5  # z-score each pixel separately
npix = beh.shape[0]

lam = .0001
ncomps = Sp.shape[0]
B= np.linalg.solve((beh @ beh.T + lam * np.eye(npix)),  (beh @ Sp.T)) # get the receptive fields for each neuron
print(beh.shape)
print(Sp.shape)
Vpred = np.matmul(B.T, beh)

In [ ]:
print(beh_reg.shape)
print(Vpred.shape)
corr_lst=[]
for j in range(0,200):
    print(np.corrcoef(Vpred[j,:],Sp[j,:])[0,1])
    corr_lst.append(np.corrcoef(Vpred[j,:],Sp[j,:])[0,1])

In [ ]:
plt.figure(figsize=(30, 15))
rfmax = np.max(B0)
for j in range(200):
    plt.subplot(10,20,j+1)
    rf = B0[:,:,j]
    # rfmax = np.max(np.abs(rf))
    plt.imshow(rf, aspect='auto', cmap = 'bwr', vmin = -rfmax, vmax = rfmax) # plot the receptive field for each neuron
    plt.title((str(corr_lst[j])[0:4]))
    plt.axis('off')
    
plt.show()    

In [ ]:
nsegs = 20
nt=NT
nlen  = nt/nsegs
ninds = np.linspace(0,nt-nlen,nsegs).astype(int)
itest = (ninds[:,np.newaxis] + np.arange(0,nlen*0.25,1,int)).flatten()
itrain = np.ones(nt, np.bool)
itrain[itest] = 0

In [ ]:
proc = np.load('/home/flora/Documents/cam1_TX39_2019_05_31_1_proc_resampled.npy', allow_pickle=True).item()
motSVD = proc['motSVD'][:,iframe+dt]
print(motSVD.shape)
beh = stats.zscore(beh, axis=1)/NT**.5  # z-score each pixel separately
npix = beh.shape[0]

lam = .0001
ncomps = Sp.shape[0]
B= np.linalg.solve((beh[:,itrain] @ beh[:,itrain].T + lam * np.eye(npix)),  (beh[:,itrain] @ Sp[:,itrain].T)) # get the receptive fields for each neuron
print(beh.shape)
print(Sp.shape)
Vpred = np.matmul(B.T, beh[:,itest])
print(Vpred.shape)

In [ ]:
print(beh_reg.shape)
print(Vpred.shape)
corr_lst=[]
for j in range(0,200):
    print(np.corrcoef(Vpred[j,:],Sp[j,itest])[0,1])
    corr_lst.append(np.corrcoef(Vpred[j,:],Sp[j,itest])[0,1])

In [ ]:
plt.figure(figsize=(30, 15))
rfmax = np.max(B0)
for j in range(200):
    plt.subplot(10,20,j+1)
    rf = B0[:,:,j]
    # rfmax = np.max(np.abs(rf))
    plt.imshow(rf, aspect='auto', cmap = 'bwr', vmin = -rfmax, vmax = rfmax) # plot the receptive field for each neuron
    plt.title((str(corr_lst[j])[0:4]))
    plt.axis('off')
    
plt.show()    

In [ ]:
print(B0.shape)
for_max=B0.reshape(12*36,200)
print(np.max(for_max,axis=0))
maximum_vis=np.max(for_max,axis=0)
plt.scatter(maximum_vis,corr_lst)

In [ ]:
from scipy.stats import zscore
Sp = V.T
Sp = zscore(Sp, axis=1)
ly, lx, nstim = mov.shape

NN, NT = S.shape 

X = np.reshape(mov, [-1, NT]) # reshape to Npixels by Ntimepoints
X = X-0.5 # subtract the background
X = np.abs(X) # does not matter if a pixel is black (0) or white (1)
X = stats.zscore(X, axis=1)/NT**.5  # z-score each pixel separately
npix = X.shape[0]

lam = .0001
ncomps = Sp.shape[0]
vis=np.linalg.solve((X @ X.T + lam * np.eye(npix)),  (X @ Sp.T))
print(vis.shape)